# A Crash Course In PySpark - Udemy

# Latest Config Code

The latest config code for Lecture 6 is included below. This can change when new versions of Spark are released. The following video covers off a similar config script (related to a previous version of Spark). However, it covers the main concepts of this code snippet to help you understand it's purpose.

In [1]:
# Latest Config Code

!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,472 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,108 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-upda

In [2]:
!ls

original.csv  sample_data  spark-3.1.1-bin-hadoop3.2  spark-3.1.1-bin-hadoop3.2.tgz


In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

import findspark
findspark.init()

from pyspark.sql import SparkSession
# spark = SparkSession.builder.getOrCreate()
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

# Ingesting & Cleaning Data

In [4]:
# 1. Upload data file
# "/content/original.csv"

# 2. read data
mydata = spark.read.format("csv").option("header", "true").load("/content/original.csv")
mydata.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  7|     Masha|    Divers|Female|         Dachun|     

In [5]:
from pyspark.sql.functions import *

# add new column 'clean_city' with null replace with 'Unknown'
mydata2 = mydata.withColumn("clean_city", when(mydata.City.isNull(), 'Unknown').otherwise(mydata.City))
mydata2.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|        Unknown|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|      Mytishchi|
|  6|     Maris|      Folk|Femal

In [6]:
# filter rows from 'JobTitle' column which is not null
mydata2 = mydata2.filter(mydata2.JobTitle.isNotNull())
mydata2.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|      Mytishchi|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|Kinsealy-Drinan|
|  8|   Goddart|     Flear|  Mal

In [7]:
# replace null values from 'Salary' column with mean
# but do some preprocessing first

# to remove '$' sign take sub-string from 2nd place to nth place and type cast it to float
mydata2 = mydata2.withColumn("clean_salary", mydata2.Salary.substr(2,100).cast("float"))
mydata2.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|clean_salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|    57438.18|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|     62846.6|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|    61489.23|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|      Mytishchi|    63863.09|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil 

In [8]:
mean = mydata2.groupBy().avg("clean_salary").take(1)[0][0]
print(mean)

55516.32088199837


In [9]:
from pyspark.sql.functions import lit # to use the litral value

# replace null with litral value of mean
mydata2 = mydata2.withColumn("new_salary", when(mydata2.clean_salary.isNull(), lit(mean)).otherwise(mydata2.clean_salary))
mydata2.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+----------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|clean_salary|      new_salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+----------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|    57438.18|   57438.1796875|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|     62846.6|   62846.6015625|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|    61489.23|  61489.23046875|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 3

In [10]:
# repalce null value from 'Latitude' and 'Longitude' column with median
import numpy as np

latitudes = mydata2.select("Latitude")
latitudes.show()

+----------+
|  Latitude|
+----------+
|50.5774075|
|48.8231572|
|44.5047212|
|      null|
|53.4266145|
|45.1905186|
| 32.027934|
|  4.272793|
|     -5.85|
| 39.172378|
|49.8151822|
|42.1014803|
|49.7923299|
|43.4945737|
|52.7441662|
| 38.696249|
|-7.7232567|
|40.7172049|
|  49.16291|
|40.7576842|
+----------+
only showing top 20 rows



In [11]:
latitudes = latitudes.filter(latitudes.Latitude.isNotNull())
latitudes.show()

+----------+
|  Latitude|
+----------+
|50.5774075|
|48.8231572|
|44.5047212|
|53.4266145|
|45.1905186|
| 32.027934|
|  4.272793|
|     -5.85|
| 39.172378|
|49.8151822|
|42.1014803|
|49.7923299|
|43.4945737|
|52.7441662|
| 38.696249|
|-7.7232567|
|40.7172049|
|  49.16291|
|40.7576842|
|48.4902808|
+----------+
only showing top 20 rows



In [12]:
latitudes = latitudes.withColumn("latitude2", latitudes.Latitude.cast("float")).select("latitude2")
latitudes.show()

+----------+
| latitude2|
+----------+
| 50.577408|
|  48.82316|
| 44.504723|
| 53.426613|
| 45.190517|
| 32.027935|
|  4.272793|
|     -5.85|
|  39.17238|
|  49.81518|
|  42.10148|
|  49.79233|
| 43.494576|
| 52.744167|
| 38.696247|
|-7.7232566|
| 40.717205|
|  49.16291|
| 40.757683|
|  48.49028|
+----------+
only showing top 20 rows



In [13]:
median = np.median(latitudes.collect())
print(median)

31.93397331237793


In [14]:
mydata2 = mydata2.withColumn("lat", when(mydata2.Latitude.isNull(), lit(median)).otherwise(mydata2.Latitude))
mydata2.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+----------------+-----------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|clean_salary|      new_salary|              lat|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+----------------+-----------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|    57438.18|   57438.1796875|       50.5774075|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|     62846.6|   62846.6015625|       48.8231572|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|    61489.23|  61489.23046875|     

In [15]:
# Answering our scenario questions
import pyspark.sql.functions as sqlfunc

# 1. Avg. salary based on genders
genders = mydata2.groupBy("gender").agg(sqlfunc.avg("new_salary").alias('AvgSalary'))
genders.show()

+------+------------------+
|gender|         AvgSalary|
+------+------------------+
|Female|55677.250125558036|
|  Male| 55361.09385573019|
+------+------------------+



In [16]:
# 2. Avg. salary based on JobTitle
departments = mydata2.groupBy("JobTitle").agg(sqlfunc.avg("new_salary").alias('AvgSalary'))
departments.show()

+--------------------+------------------+
|            JobTitle|         AvgSalary|
+--------------------+------------------+
|Systems Administr...|  66131.4248046875|
|   Media Manager III|46968.356770833336|
|  Recruiting Manager|61231.947591145836|
|       Geologist III|    44579.80078125|
|        Geologist II|   43293.865234375|
|Database Administ...|     52018.4609375|
|   Financial Analyst|   62959.833203125|
|  Analyst Programmer|     37449.0921875|
|Software Engineer II|      74782.640625|
|       Accountant IV|   82732.248046875|
|    Product Engineer|    62290.42734375|
|Software Test Eng...|  59341.0712890625|
|Safety Technician...|   29421.529296875|
|    Junior Executive|    65262.55078125|
|Systems Administr...|       77059.21875|
|Human Resources A...| 40631.93994140625|
|        VP Marketing|60825.713216145836|
|  Environmental Tech|59367.870768229164|
|Mechanical System...|         75692.375|
| Assistant Professor|    49088.72421875|
+--------------------+------------

In [17]:
# 3. Avg. salary based on City
cities = mydata2.groupBy("City").agg(sqlfunc.avg("new_salary").alias('AvgSalary'))
cities.show()

+-----------------+----------------+
|             City|       AvgSalary|
+-----------------+----------------+
|        Sułkowice|  33432.98828125|
|          Klippan|     77039.46875|
|      Trollhättan|53311.6845703125|
|        Shinaihai|    39544.640625|
|         Hongzhou|  35707.30859375|
|         Cipinang| 11617.509765625|
| Viejo Daan Banua|         43927.5|
|         Tsiatsan| 18795.439453125|
|       San Andres|  52426.80078125|
|           Krasna|   72022.7890625|
|      Springfield|40697.3251953125|
|            Město|  27797.98046875|
|Chaloem Phra Kiat|  54840.19921875|
|          Tadotsu|  55595.30078125|
|   Hénin-Beaumont|        55082.75|
|          Kajaani|  20224.83984375|
|           Duozhu|    71416.859375|
|           Abéché|   93375.1796875|
|     Habingkloang|     56892.96875|
|         Malishka|   76783.4765625|
+-----------------+----------------+
only showing top 20 rows



In [18]:
# 4. find avg male and female salary w.r.t. job title
# df = mydata2.groupBy("JobTitle", "gender").agg(sqlfunc.avg("new_salary").alias('AvgSalary'))
# df.show()

df = mydata2.withColumn("female_salary", when(mydata2.gender == 'Female', mydata2.new_salary).otherwise(lit(0)))
df = df.withColumn("male_salary", when(mydata2.gender == 'Male', mydata2.new_salary).otherwise(lit(0)))

df = df.groupBy("JobTitle").agg(sqlfunc.avg("female_salary").alias('avg_female_salary'),
                                sqlfunc.avg("male_salary").alias('avg_male_salary'))
df.show()

+--------------------+------------------+------------------+
|            JobTitle| avg_female_salary|   avg_male_salary|
+--------------------+------------------+------------------+
|Systems Administr...|   50590.474609375|  15540.9501953125|
|   Media Manager III|29586.436197916668|17381.920572916668|
|  Recruiting Manager|34848.452473958336|  26383.4951171875|
|       Geologist III|      31749.046875|    12830.75390625|
|        Geologist II|               0.0|   43293.865234375|
|Database Administ...|               0.0|     52018.4609375|
|   Financial Analyst|   23353.776953125|       39606.05625|
|  Analyst Programmer|  16406.1287109375|  21042.9634765625|
|Software Engineer II|               0.0|      74782.640625|
|       Accountant IV|   82732.248046875|               0.0|
|    Product Engineer|    41825.48359375|       20464.94375|
|Software Test Eng...|  32218.6083984375|   27122.462890625|
|Safety Technician...|               0.0|   29421.529296875|
|    Junior Executive|15

In [19]:
# 5. find delta between male and female salary
df = df.withColumn("delta", df.avg_female_salary - df.avg_male_salary)
df.show()

+--------------------+------------------+------------------+-------------------+
|            JobTitle| avg_female_salary|   avg_male_salary|              delta|
+--------------------+------------------+------------------+-------------------+
|Systems Administr...|   50590.474609375|  15540.9501953125|   35049.5244140625|
|   Media Manager III|29586.436197916668|17381.920572916668|       12204.515625|
|  Recruiting Manager|34848.452473958336|  26383.4951171875|  8464.957356770836|
|       Geologist III|      31749.046875|    12830.75390625|     18918.29296875|
|        Geologist II|               0.0|   43293.865234375|   -43293.865234375|
|Database Administ...|               0.0|     52018.4609375|     -52018.4609375|
|   Financial Analyst|   23353.776953125|       39606.05625|   -16252.279296875|
|  Analyst Programmer|  16406.1287109375|  21042.9634765625| -4636.834765625001|
|Software Engineer II|               0.0|      74782.640625|      -74782.640625|
|       Accountant IV|   827

In [20]:
# 6. which city has highest Salary
cityavg = mydata2.groupBy("City").agg(sqlfunc.avg("new_salary").alias('AvgSalary'))
cityavg = cityavg.sort(cityavg.AvgSalary.desc())
cityavg.show()

+-----------------+-------------+
|             City|    AvgSalary|
+-----------------+-------------+
|        Mesopotam|  99948.28125|
|       Zhongcheng| 99942.921875|
|           Caxias|99786.3984375|
|      Karangtawar|99638.9921875|
|        Itabaiana|  99502.15625|
|           Pasian|  99421.34375|
|           Webuye| 99368.546875|
|      Yuktae-dong| 99250.828125|
|           Zinder|  99222.84375|
|   Timiryazevskiy|   99142.9375|
|        Sawahbaru|99013.7109375|
|          Madimba|98737.8671875|
|         Huangshi|  98690.34375|
|          Gharyan|   98679.3125|
|         Yŏnan-ŭp| 98628.609375|
|     Wringinputih|98603.8203125|
|Monte da Boavista|  98586.71875|
|          Klukeng|98439.4921875|
|         Murmashi|  98226.15625|
|        Fox Creek|      98138.0|
+-----------------+-------------+
only showing top 20 rows



# Course content

# Bringing data into dataframes

In [21]:
df2 = spark.read.csv("/content/original.csv", header=True)
df2.show()

# other file formate are simple. eg. spark.read.text(....)

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  7|     Masha|    Divers|Female|         Dachun|     

In [22]:
df2.dtypes

[('id', 'string'),
 ('first_name', 'string'),
 ('last_name', 'string'),
 ('gender', 'string'),
 ('City', 'string'),
 ('JobTitle', 'string'),
 ('Salary', 'string'),
 ('Latitude', 'string'),
 ('Longitude', 'string')]

In [23]:
from pyspark.sql.types import *

schema = StructType([
    StructField("id", IntegerType()),
    StructField("first_name", StringType()),
    StructField("last_name", StringType()),
    StructField("gender", StringType()),
    StructField("City", StringType()),
    StructField("JobTitle", StringType()),
    StructField("Salary", StringType()),
    StructField("Latitude", FloatType()),
    StructField("Longitude", FloatType())
])

df3 = spark.read.csv("/content/original.csv", header=True, schema=schema)
df3.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude| Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18| 50.577408| 16.496717|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|  48.82316| 103.52182|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52| 39.994747|116.339775|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23| 44.504723| 38.130016|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.648994|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16| 53.426613|-6.1644998|
|  7|     Masha|    Divers|Female|         Dachun|              

In [24]:
df3.dtypes

[('id', 'int'),
 ('first_name', 'string'),
 ('last_name', 'string'),
 ('gender', 'string'),
 ('City', 'string'),
 ('JobTitle', 'string'),
 ('Salary', 'string'),
 ('Latitude', 'float'),
 ('Longitude', 'float')]

# Inspecting A Dataframe

In [25]:
df3.dtypes

[('id', 'int'),
 ('first_name', 'string'),
 ('last_name', 'string'),
 ('gender', 'string'),
 ('City', 'string'),
 ('JobTitle', 'string'),
 ('Salary', 'string'),
 ('Latitude', 'float'),
 ('Longitude', 'float')]

In [26]:
df3.head(6)

[Row(id=1, first_name='Melinde', last_name='Shilburne', gender='Female', City='Nowa Ruda', JobTitle='Assistant Professor', Salary='$57438.18', Latitude=50.57740783691406, Longitude=16.49671745300293),
 Row(id=2, first_name='Kimberly', last_name='Von Welden', gender='Female', City='Bulgan', JobTitle='Programmer II', Salary='$62846.60', Latitude=48.823158264160156, Longitude=103.52182006835938),
 Row(id=3, first_name='Alvera', last_name='Di Boldi', gender='Female', City=None, JobTitle=None, Salary='$57576.52', Latitude=39.994747161865234, Longitude=116.33977508544922),
 Row(id=4, first_name='Shannon', last_name="O'Griffin", gender='Male', City='Divnomorskoye', JobTitle='Budget/Accounting Analyst II', Salary='$61489.23', Latitude=44.504722595214844, Longitude=38.1300163269043),
 Row(id=5, first_name='Sherwood', last_name='Macieja', gender='Male', City='Mytishchi', JobTitle='VP Sales', Salary='$63863.09', Latitude=None, Longitude=37.64899444580078),
 Row(id=6, first_name='Maris', last_name

In [27]:
df3.first()

Row(id=1, first_name='Melinde', last_name='Shilburne', gender='Female', City='Nowa Ruda', JobTitle='Assistant Professor', Salary='$57438.18', Latitude=50.57740783691406, Longitude=16.49671745300293)

In [29]:
df3.describe().show()

+-------+-----------------+----------+---------+------+-------------------+-------------------+---------+------------------+-----------------+
|summary|               id|first_name|last_name|gender|               City|           JobTitle|   Salary|          Latitude|        Longitude|
+-------+-----------------+----------+---------+------+-------------------+-------------------+---------+------------------+-----------------+
|  count|             1000|      1000|     1000|  1000|                999|                998|     1000|               999|             1000|
|   mean|            500.5|      null|     null|  null|               null|               null|     null| 25.43151724702484|43.33756460386515|
| stddev|288.8194360957494|      null|     null|  null|               null|               null|     null|24.579082550156635| 69.4206453674681|
|    min|                1|   Abagail|    Abbay|Female|             Abéché|Account Coordinator|$10101.92|         -54.28115|       -123.04196|

In [30]:
df3.columns

['id',
 'first_name',
 'last_name',
 'gender',
 'City',
 'JobTitle',
 'Salary',
 'Latitude',
 'Longitude']

In [31]:
df3.count()

1000

In [32]:
df3.distinct().count()

1000

# Handling Null & Duplicate Values

In [35]:
from pyspark.sql.types import *

schema = StructType([
    StructField("id", IntegerType()),
    StructField("first_name", StringType()),
    StructField("last_name", StringType()),
    StructField("gender", StringType()),
    StructField("City", StringType()),
    StructField("JobTitle", StringType()),
    StructField("Salary", StringType()),
    StructField("Latitude", FloatType()),
    StructField("Longitude", FloatType())
])

df = spark.read.csv("/content/original.csv", header=True, schema=schema)
df.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude| Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18| 50.577408| 16.496717|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|  48.82316| 103.52182|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52| 39.994747|116.339775|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23| 44.504723| 38.130016|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.648994|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16| 53.426613|-6.1644998|
|  7|     Masha|    Divers|Female|         Dachun|              

In [36]:
# drop na values
df_dropped = df.na.drop()
df_dropped.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude| Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18| 50.577408| 16.496717|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|  48.82316| 103.52182|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23| 44.504723| 38.130016|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16| 53.426613|-6.1644998|
|  8|   Goddart|     Flear|  Male|      Trélissac|Desktop Support T...|$46116.36| 45.190517| 0.7423124|
|  9|      Roth|O'Cannavan|  Male|         Heitan|VP Product Manage...|$73697.10| 32.027935| 106.65711|
| 10|      Bran|   Trahear|  Male|       Arbeláez|Mechanical Sys

In [37]:
# drop na values from one column
df_not_null_jobs = df.na.drop(subset=["JobTitle"])
df_not_null_jobs.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude| Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18| 50.577408| 16.496717|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|  48.82316| 103.52182|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23| 44.504723| 38.130016|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.648994|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16| 53.426613|-6.1644998|
|  8|   Goddart|     Flear|  Male|      Trélissac|Desktop Support T...|$46116.36| 45.190517| 0.7423124|
|  9|      Roth|O'Cannavan|  Male|         Heitan|VP Product Man

In [38]:
# same by using filter()
df_not_null_jobs = df.filter(df.JobTitle.isNotNull())
df_not_null_jobs.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude| Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18| 50.577408| 16.496717|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|  48.82316| 103.52182|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23| 44.504723| 38.130016|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.648994|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16| 53.426613|-6.1644998|
|  8|   Goddart|     Flear|  Male|      Trélissac|Desktop Support T...|$46116.36| 45.190517| 0.7423124|
|  9|      Roth|O'Cannavan|  Male|         Heitan|VP Product Man

In [41]:
# creating a new column
from pyspark.sql.functions import *

df_new = df.withColumn("new_salary", df.Salary.substr(2,100).cast("float") * 2)
df_new.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+----------+----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude| Longitude|new_salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+----------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18| 50.577408| 16.496717| 114876.36|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|  48.82316| 103.52182|  125693.2|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52| 39.994747|116.339775| 115153.04|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23| 44.504723| 38.130016| 122978.46|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.648994| 127726.18|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16

In [42]:
df_handled = df.withColumn("clean_city", when(df.City.isNull(), 'Unknown').otherwise(df.City))
df_handled.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+----------+---------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude| Longitude|     clean_city|
+---+----------+----------+------+---------------+--------------------+---------+----------+----------+---------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18| 50.577408| 16.496717|      Nowa Ruda|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|  48.82316| 103.52182|         Bulgan|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52| 39.994747|116.339775|        Unknown|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23| 44.504723| 38.130016|  Divnomorskoye|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.648994|      Mytishchi|
|  6|     Maris|      Folk|Female|Kinsea

In [43]:
# drop duplicates
df_no_duplicates = df.dropDuplicates()
df_no_duplicates.show()

+---+----------+-------------+------+--------------------+--------------------+---------+----------+----------+
| id|first_name|    last_name|gender|                City|            JobTitle|   Salary|  Latitude| Longitude|
+---+----------+-------------+------+--------------------+--------------------+---------+----------+----------+
|372|     Lyman|      Burfitt|  Male|             Guiping|Community Outreac...|$28755.53| 23.394325| 110.07938|
|391|     Verge|     Hefferan|  Male|          Cocachacra|Community Outreac...|$90391.71|-17.091843| -71.77114|
|428|   Pernell|      Fossitt|  Male|           Wangchang|    Graphic Designer|$36927.53| 40.401047|  117.9989|
|526|  Garfield|    Benadette|  Male|              Shiren|   Marketing Manager|$56867.78| 28.651703| 117.90413|
|731|     Legra|        Manns|Female|               Nîmes|  Research Associate|$42246.87| 43.844727| 4.3520436|
|809|       Jed|       Shires|  Male|            Goubétto|     Design Engineer|$64130.79| 11.423197| 43.

# Selecting & Filtering Data

In [44]:
from pyspark.sql.types import *

schema = StructType([
    StructField("id", IntegerType()),
    StructField("first_name", StringType()),
    StructField("last_name", StringType()),
    StructField("gender", StringType()),
    StructField("City", StringType()),
    StructField("JobTitle", StringType()),
    StructField("Salary", StringType()),
    StructField("Latitude", FloatType()),
    StructField("Longitude", FloatType())
])

df = spark.read.csv("/content/original.csv", header=True, schema=schema)
df.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude| Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18| 50.577408| 16.496717|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|  48.82316| 103.52182|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52| 39.994747|116.339775|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23| 44.504723| 38.130016|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.648994|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16| 53.426613|-6.1644998|
|  7|     Masha|    Divers|Female|         Dachun|              

In [45]:
# select 3 columns only
df_select = df.select("first_name", "last_name")
df_select.show()

+----------+----------+
|first_name| last_name|
+----------+----------+
|   Melinde| Shilburne|
|  Kimberly|Von Welden|
|    Alvera|  Di Boldi|
|   Shannon| O'Griffin|
|  Sherwood|   Macieja|
|     Maris|      Folk|
|     Masha|    Divers|
|   Goddart|     Flear|
|      Roth|O'Cannavan|
|      Bran|   Trahear|
|    Kylynn|   Lockart|
|       Rey|    Meharg|
|      Kerr|    Braden|
|    Mickie| Whanstall|
|    Kaspar|     Pally|
|    Norbie|    Gwyllt|
|    Claude|    Briant|
|     Thain|    Habbon|
|  Tiffanie|  Pattison|
|    Ettore|  Gerriets|
+----------+----------+
only showing top 20 rows



In [46]:
# rename the column
df_renamed = df.withColumnRenamed("first_name", "fn")
df_renamed.show()

+---+--------+----------+------+---------------+--------------------+---------+----------+----------+
| id|      fn| last_name|gender|           City|            JobTitle|   Salary|  Latitude| Longitude|
+---+--------+----------+------+---------------+--------------------+---------+----------+----------+
|  1| Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18| 50.577408| 16.496717|
|  2|Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|  48.82316| 103.52182|
|  3|  Alvera|  Di Boldi|Female|           null|                null|$57576.52| 39.994747|116.339775|
|  4| Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23| 44.504723| 38.130016|
|  5|Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.648994|
|  6|   Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16| 53.426613|-6.1644998|
|  7|   Masha|    Divers|Female|         Dachun|                null|$25090.87| 24

In [48]:
# filter by first name
df_filtered = df.filter(df.first_name == 'Rey')
df_filtered.show()

+---+----------+---------+------+-----------+--------------------+---------+--------+---------+
| id|first_name|last_name|gender|       City|            JobTitle|   Salary|Latitude|Longitude|
+---+----------+---------+------+-----------+--------------------+---------+--------+---------+
| 12|       Rey|   Meharg|Female|Wangqingtuo|Systems Administr...|$73423.70|39.17238|116.93161|
|925|       Rey|  Girardy|  Male|   Malishka|   Financial Analyst|$76783.48|39.73761|45.392582|
+---+----------+---------+------+-----------+--------------------+---------+--------+---------+



In [50]:
df_filtered = df.filter(df.first_name.like("%vera"))
df_filtered.show()

+---+----------+---------+------+----+--------+---------+---------+----------+
| id|first_name|last_name|gender|City|JobTitle|   Salary| Latitude| Longitude|
+---+----------+---------+------+----+--------+---------+---------+----------+
|  3|    Alvera| Di Boldi|Female|null|    null|$57576.52|39.994747|116.339775|
+---+----------+---------+------+----+--------+---------+---------+----------+



In [54]:
df_filtered = df.filter(df.first_name.like("%ra%"))
df_filtered.show()

+---+----------+----------+------+----------+--------------------+---------+----------+----------+
| id|first_name| last_name|gender|      City|            JobTitle|   Salary|  Latitude| Longitude|
+---+----------+----------+------+----------+--------------------+---------+----------+----------+
|  3|    Alvera|  Di Boldi|Female|      null|                null|$57576.52| 39.994747|116.339775|
| 10|      Bran|   Trahear|  Male|  Arbeláez|Mechanical System...|$68098.42|  4.272793|-74.416016|
| 26|   Leandra|    Anfrey|Female|   Isfahan|VP Product Manage...|$30201.32|  32.65463| 51.667984|
| 49|  Franklyn|     Campo|  Male|   Rzhavki|      Statistician I|$62765.00| 56.012775|  37.22298|
| 65|    Faydra|  Ashpital|Female|   Kakamas|Budget/Accounting...|$44648.27|-28.672731| 20.428444|
|129|    Franny|   Austing|  Male|   Cuogang|Community Outreac...|$78582.67| 43.494576|  5.897802|
|134|     Grant|  Tilbrook|  Male|   Cuogang|Safety Technician...|$29421.53|  59.33624| 18.023237|
|177|   Ge

In [51]:
df_filtered = df.filter(df.first_name.endswith("din"))
df_filtered.show()

+---+----------+-------------+------+-----------+--------+---------+----------+---------+
| id|first_name|    last_name|gender|       City|JobTitle|   Salary|  Latitude|Longitude|
+---+----------+-------------+------+-----------+--------+---------+----------+---------+
|901|     Aldin|Matuszkiewicz|  Male|East London|Operator|$41468.83|-32.954933|27.931913|
+---+----------+-------------+------+-----------+--------+---------+----------+---------+



In [56]:
df_filtered = df.filter(df.first_name.startswith("Bi"))
df_filtered.show()

+---+----------+---------+------+----------+--------------------+---------+---------+---------+
| id|first_name|last_name|gender|      City|            JobTitle|   Salary| Latitude|Longitude|
+---+----------+---------+------+----------+--------------------+---------+---------+---------+
|247|    Binnie|Concannon|Female|Sungai Iyu|Business Systems ...|$80136.29| 4.405403|   98.168|
|416|     Birch|   Foltin|  Male|      Dubá|Chief Design Engi...|$28119.28| 50.53457|14.539722|
|798|     Biron| Richardt|  Male|  El Monte|    Graphic Designer|$85623.28|-33.68153| -70.9854|
|834|    Binnie|  Neligan|Female|  Longgang|   Director of Sales|$34565.59|22.720974| 114.2469|
+---+----------+---------+------+----------+--------------------+---------+---------+---------+



In [58]:
df_filtered = df.filter(df.id.between(10,15))
df_filtered.show()

+---+----------+---------+------+-----------+--------------------+---------+--------+----------+
| id|first_name|last_name|gender|       City|            JobTitle|   Salary|Latitude| Longitude|
+---+----------+---------+------+-----------+--------------------+---------+--------+----------+
| 10|      Bran|  Trahear|  Male|   Arbeláez|Mechanical System...|$68098.42|4.272793|-74.416016|
| 11|    Kylynn|  Lockart|Female|   El Cardo|Nuclear Power Eng...|$13604.63|   -5.85| -79.88333|
| 12|       Rey|   Meharg|Female|Wangqingtuo|Systems Administr...|$73423.70|39.17238| 116.93161|
| 13|      Kerr|   Braden|  Male|  Sułkowice|Compensation Analyst|$33432.99|49.81518| 19.377174|
| 14|    Mickie|Whanstall|  Male|Springfield|Assistant Media P...|$50838.53|42.10148|-72.576675|
| 15|    Kaspar|    Pally|  Male|     Chrást|  Analyst Programmer|$40163.03|49.79233| 13.491532|
+---+----------+---------+------+-----------+--------------------+---------+--------+----------+



In [59]:
df_filtered = df.filter(df.first_name.isin('Rey', 'Bran'))
df_filtered.show()

+---+----------+---------+------+-----------+--------------------+---------+--------+----------+
| id|first_name|last_name|gender|       City|            JobTitle|   Salary|Latitude| Longitude|
+---+----------+---------+------+-----------+--------------------+---------+--------+----------+
| 10|      Bran|  Trahear|  Male|   Arbeláez|Mechanical System...|$68098.42|4.272793|-74.416016|
| 12|       Rey|   Meharg|Female|Wangqingtuo|Systems Administr...|$73423.70|39.17238| 116.93161|
|925|       Rey|  Girardy|  Male|   Malishka|   Financial Analyst|$76783.48|39.73761| 45.392582|
+---+----------+---------+------+-----------+--------------------+---------+--------+----------+



In [65]:
df_substr = df.select(df.first_name, lower(df.first_name.substr(1,5)).alias("sub_name"))
df_substr.show()

+----------+--------+
|first_name|sub_name|
+----------+--------+
|   Melinde|   melin|
|  Kimberly|   kimbe|
|    Alvera|   alver|
|   Shannon|   shann|
|  Sherwood|   sherw|
|     Maris|   maris|
|     Masha|   masha|
|   Goddart|   godda|
|      Roth|    roth|
|      Bran|    bran|
|    Kylynn|   kylyn|
|       Rey|     rey|
|      Kerr|    kerr|
|    Mickie|   micki|
|    Kaspar|   kaspa|
|    Norbie|   norbi|
|    Claude|   claud|
|     Thain|   thain|
|  Tiffanie|   tiffa|
|    Ettore|   ettor|
+----------+--------+
only showing top 20 rows



# Applying Multiple Filters

In [68]:
df_filtered = df.filter(
    (df.first_name.isin('Rey', 'Bran'))
    &
    (df.id.between(10,15))
    )
df_filtered.show()

+---+----------+---------+------+-----------+--------------------+---------+--------+----------+
| id|first_name|last_name|gender|       City|            JobTitle|   Salary|Latitude| Longitude|
+---+----------+---------+------+-----------+--------------------+---------+--------+----------+
| 10|      Bran|  Trahear|  Male|   Arbeláez|Mechanical System...|$68098.42|4.272793|-74.416016|
| 12|       Rey|   Meharg|Female|Wangqingtuo|Systems Administr...|$73423.70|39.17238| 116.93161|
+---+----------+---------+------+-----------+--------------------+---------+--------+----------+



In [71]:
df_filtered = df.filter(
    (df.first_name.isin('Aldin', 'Bran'))
    &
    (df.City.like('%London')) # case sensitive
    )
df_filtered.show()

+---+----------+-------------+------+-----------+--------+---------+----------+---------+
| id|first_name|    last_name|gender|       City|JobTitle|   Salary|  Latitude|Longitude|
+---+----------+-------------+------+-----------+--------+---------+----------+---------+
|901|     Aldin|Matuszkiewicz|  Male|East London|Operator|$41468.83|-32.954933|27.931913|
+---+----------+-------------+------+-----------+--------+---------+----------+---------+



In [73]:
df_filtered = df.filter((df.id > 10) & (df.id < 20))
df_filtered.show()

+---+----------+---------+------+--------------+--------------------+---------+----------+----------+
| id|first_name|last_name|gender|          City|            JobTitle|   Salary|  Latitude| Longitude|
+---+----------+---------+------+--------------+--------------------+---------+----------+----------+
| 11|    Kylynn|  Lockart|Female|      El Cardo|Nuclear Power Eng...|$13604.63|     -5.85| -79.88333|
| 12|       Rey|   Meharg|Female|   Wangqingtuo|Systems Administr...|$73423.70|  39.17238| 116.93161|
| 13|      Kerr|   Braden|  Male|     Sułkowice|Compensation Analyst|$33432.99|  49.81518| 19.377174|
| 14|    Mickie|Whanstall|  Male|   Springfield|Assistant Media P...|$50838.53|  42.10148|-72.576675|
| 15|    Kaspar|    Pally|  Male|        Chrást|  Analyst Programmer|$40163.03|  49.79233| 13.491532|
| 16|    Norbie|   Gwyllt|  Male|        Xijiao|              Editor|$32492.73| 43.494576|  5.897802|
| 17|    Claude|   Briant|Female|     Mieścisko|Research Assistan...|$51862.48| 52

In [74]:
df_filtered = df.filter(
    (df.first_name.isin('Aldin', 'Bran'))
    |
    (df.City.like('%London')) # case sensitive
    )
df_filtered.show()

+---+----------+-------------+------+-----------+--------------------+---------+----------+----------+
| id|first_name|    last_name|gender|       City|            JobTitle|   Salary|  Latitude| Longitude|
+---+----------+-------------+------+-----------+--------------------+---------+----------+----------+
| 10|      Bran|      Trahear|  Male|   Arbeláez|Mechanical System...|$68098.42|  4.272793|-74.416016|
|901|     Aldin|Matuszkiewicz|  Male|East London|            Operator|$41468.83|-32.954933| 27.931913|
+---+----------+-------------+------+-----------+--------------------+---------+----------+----------+



# Running SQL on Dataframes

In [75]:
from pyspark.sql.types import *

schema = StructType([
    StructField("id", IntegerType()),
    StructField("first_name", StringType()),
    StructField("last_name", StringType()),
    StructField("gender", StringType()),
    StructField("City", StringType()),
    StructField("JobTitle", StringType()),
    StructField("Salary", StringType()),
    StructField("Latitude", FloatType()),
    StructField("Longitude", FloatType())
])

df = spark.read.csv("/content/original.csv", header=True, schema=schema)
df.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude| Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18| 50.577408| 16.496717|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|  48.82316| 103.52182|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52| 39.994747|116.339775|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23| 44.504723| 38.130016|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.648994|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16| 53.426613|-6.1644998|
|  7|     Masha|    Divers|Female|         Dachun|              

In [76]:
# create a temp table to run the sql queries
# df.createOrReplaceTempView("original")
df.registerTempTable("original")

In [77]:
query1 = spark.sql('select * from original')
query1.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude| Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18| 50.577408| 16.496717|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|  48.82316| 103.52182|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52| 39.994747|116.339775|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23| 44.504723| 38.130016|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.648994|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16| 53.426613|-6.1644998|
|  7|     Masha|    Divers|Female|         Dachun|              

In [79]:
%%time
query2 = spark.sql('''
    select concat(first_name, ' ', last_name) as full_name
    from original
    where gender = 'Female'
    ''')
query2.show()

+-------------------+
|          full_name|
+-------------------+
|  Melinde Shilburne|
|Kimberly Von Welden|
|    Alvera Di Boldi|
|         Maris Folk|
|       Masha Divers|
|     Kylynn Lockart|
|         Rey Meharg|
|      Claude Briant|
|  Tiffanie Pattison|
|    Lurleen Janczak|
|      Nichol Holtum|
|       Shaun Bridle|
|     Leandra Anfrey|
|    Jaquelyn Hazard|
|  Prudence Honacker|
|       Cherey Liger|
|          Neda Krop|
|    Barbi Fattorini|
|   Lonnie Townshend|
|    Valida Salzberg|
+-------------------+
only showing top 20 rows

CPU times: user 618 µs, sys: 2.06 ms, total: 2.68 ms
Wall time: 104 ms


# Adding Calculated Columns

In [80]:
from pyspark.sql.types import *

schema = StructType([
    StructField("id", IntegerType()),
    StructField("first_name", StringType()),
    StructField("last_name", StringType()),
    StructField("gender", StringType()),
    StructField("City", StringType()),
    StructField("JobTitle", StringType()),
    StructField("Salary", StringType()),
    StructField("Latitude", FloatType()),
    StructField("Longitude", FloatType())
])

df = spark.read.csv("/content/original.csv", header=True, schema=schema)
df.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude| Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18| 50.577408| 16.496717|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|  48.82316| 103.52182|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52| 39.994747|116.339775|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23| 44.504723| 38.130016|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.648994|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16| 53.426613|-6.1644998|
|  7|     Masha|    Divers|Female|         Dachun|              

In [83]:
from pyspark.sql.functions import *
df = df.withColumn('new_Salary', df.Salary.substr(2,100).cast('float'))
df.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+----------+----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude| Longitude|new_Salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+----------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18| 50.577408| 16.496717|  57438.18|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|  48.82316| 103.52182|   62846.6|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52| 39.994747|116.339775|  57576.52|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23| 44.504723| 38.130016|  61489.23|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.648994|  63863.09|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16

In [84]:
df = df.withColumn('monthly_Salary', df.new_Salary / 12)
df.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+----------+----------+------------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude| Longitude|new_Salary|    monthly_Salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+----------+----------+------------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18| 50.577408| 16.496717|  57438.18| 4786.514973958333|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|  48.82316| 103.52182|   62846.6|    5237.216796875|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52| 39.994747|116.339775|  57576.52| 4798.043294270833|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23| 44.504723| 38.130016|  61489.23|   5124.1025390625|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            

In [86]:
df = df.withColumn('are_they_Female', when(df.gender == 'Female', 'Yes').otherwise('No'))
df.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+----------+----------+------------------+---------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude| Longitude|new_Salary|    monthly_Salary|are_they_Female|
+---+----------+----------+------+---------------+--------------------+---------+----------+----------+----------+------------------+---------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18| 50.577408| 16.496717|  57438.18| 4786.514973958333|            Yes|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|  48.82316| 103.52182|   62846.6|    5237.216796875|            Yes|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52| 39.994747|116.339775|  57576.52| 4798.043294270833|            Yes|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23| 44.504723| 38.1300

In [87]:
df = df.withColumn('are_they_rich', when(df.new_Salary > 50000, 'Yes').otherwise('No'))
df.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+----------+----------+------------------+---------------+-------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude| Longitude|new_Salary|    monthly_Salary|are_they_Female|are_they_rich|
+---+----------+----------+------+---------------+--------------------+---------+----------+----------+----------+------------------+---------------+-------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18| 50.577408| 16.496717|  57438.18| 4786.514973958333|            Yes|          Yes|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|  48.82316| 103.52182|   62846.6|    5237.216796875|            Yes|          Yes|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52| 39.994747|116.339775|  57576.52| 4798.043294270833|            Yes|          Yes|
|  4|   Shannon|

# Group By And Aggregation

In [88]:
from pyspark.sql.types import *

schema = StructType([
    StructField("id", IntegerType()),
    StructField("first_name", StringType()),
    StructField("last_name", StringType()),
    StructField("gender", StringType()),
    StructField("City", StringType()),
    StructField("JobTitle", StringType()),
    StructField("Salary", StringType()),
    StructField("Latitude", FloatType()),
    StructField("Longitude", FloatType())
])

df = spark.read.csv("/content/original.csv", header=True, schema=schema)
df.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude| Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18| 50.577408| 16.496717|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|  48.82316| 103.52182|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52| 39.994747|116.339775|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23| 44.504723| 38.130016|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.648994|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16| 53.426613|-6.1644998|
|  7|     Masha|    Divers|Female|         Dachun|              

In [89]:
from pyspark.sql.functions import *
df = df.withColumn('new_Salary', df.Salary.substr(2,100).cast('float'))
df.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+----------+----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude| Longitude|new_Salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+----------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18| 50.577408| 16.496717|  57438.18|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|  48.82316| 103.52182|   62846.6|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52| 39.994747|116.339775|  57576.52|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23| 44.504723| 38.130016|  61489.23|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.648994|  63863.09|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16

In [90]:
import pyspark.sql.functions as sqlfunc

df1 = df.groupBy('gender').count().show()

+------+-----+
|gender|count|
+------+-----+
|Female|  492|
|  Male|  508|
+------+-----+



In [96]:
df1 = df.groupBy('gender').agg(sqlfunc.sum('new_Salary').cast('int').alias('total_salary')).show()

+------+------------+
|gender|total_salary|
+------+------------+
|Female|    27364519|
|  Male|    28123435|
+------+------------+



In [97]:
df1 = df.groupBy('gender').agg(
    sqlfunc.sum('new_Salary').cast('int').alias('total_salary'),
    sqlfunc.avg('new_Salary').cast('int').alias('avg_salary'),
    sqlfunc.min('new_Salary').cast('int').alias('min_salary'),
    sqlfunc.max('new_Salary').cast('int').alias('max_salary')
    ).show()

+------+------------+----------+----------+----------+
|gender|total_salary|avg_salary|min_salary|max_salary|
+------+------------+----------+----------+----------+
|Female|    27364519|     55618|     10616|     99948|
|  Male|    28123435|     55361|     10101|     99942|
+------+------------+----------+----------+----------+



In [98]:
df1 = df.groupBy('gender', 'City').agg(
    sqlfunc.sum('new_Salary').cast('int').alias('total_salary'),
    sqlfunc.avg('new_Salary').cast('int').alias('avg_salary'),
    sqlfunc.min('new_Salary').cast('int').alias('min_salary'),
    sqlfunc.max('new_Salary').cast('int').alias('max_salary')
    ).show()

+------+-----------------+------------+----------+----------+----------+
|gender|             City|total_salary|avg_salary|min_salary|max_salary|
+------+-----------------+------------+----------+----------+----------+
|Female|           Dachun|       25090|     25090|     25090|     25090|
|Female|      Trollhättan|      106623|     53311|     26830|     79792|
|  Male|          Wenshao|       18941|     18941|     18941|     18941|
|Female|            Lanas|       13765|     13765|     13765|     13765|
|  Male|            Mörön|       77940|     77940|     77940|     77940|
|Female|             Same|       73369|     73369|     73369|     73369|
|Female|          Sawahan|       24608|     24608|     24608|     24608|
|  Male|Monte da Boavista|       98586|     98586|     98586|     98586|
|Female|         Nusajaya|       71637|     71637|     71637|     71637|
|Female|            Kista|       96192|     96192|     96192|     96192|
|  Male|       Pittsburgh|       83121|     83121| 

# Writing Dataframe To Files

In [100]:
df1 = df.groupBy('gender', 'City').agg(
    sqlfunc.sum('new_Salary').cast('int').alias('total_salary'),
    sqlfunc.avg('new_Salary').cast('int').alias('avg_salary'),
    sqlfunc.min('new_Salary').cast('int').alias('min_salary'),
    sqlfunc.max('new_Salary').cast('int').alias('max_salary')
    )
df1.show()

+------+-----------------+------------+----------+----------+----------+
|gender|             City|total_salary|avg_salary|min_salary|max_salary|
+------+-----------------+------------+----------+----------+----------+
|Female|           Dachun|       25090|     25090|     25090|     25090|
|Female|      Trollhättan|      106623|     53311|     26830|     79792|
|  Male|          Wenshao|       18941|     18941|     18941|     18941|
|Female|            Lanas|       13765|     13765|     13765|     13765|
|  Male|            Mörön|       77940|     77940|     77940|     77940|
|Female|             Same|       73369|     73369|     73369|     73369|
|Female|          Sawahan|       24608|     24608|     24608|     24608|
|  Male|Monte da Boavista|       98586|     98586|     98586|     98586|
|Female|         Nusajaya|       71637|     71637|     71637|     71637|
|Female|            Kista|       96192|     96192|     96192|     96192|
|  Male|       Pittsburgh|       83121|     83121| 

In [102]:
df1.write.csv('df1_csv')
df1.write.json('df1_json')
df1.write.parquet('df1_parquet')

# can add partitions and compression as options.

# Challenge

1. Add a column to say yes or no to whether the country is Mexico.

2. Group by your new column and sum bytes_used.

3. Group by country and use the sqlfunc.countDistinct function to calculate the number of IP address seen in EACH country.

In [105]:
from pyspark.sql.types import *

schema = StructType([
    StructField("ip_address", StringType()),
    StructField("Country", StringType()),
    StructField("Domain Name", StringType()),
    StructField("Bytes_used", IntegerType())
])

df = spark.read.csv("/content/challenge.csv", header=True, schema=schema)
df.show()

+---------------+--------------+-----------------+----------+
|     ip_address|       Country|      Domain Name|Bytes_used|
+---------------+--------------+-----------------+----------+
|  52.81.192.172|         China| odnoklassniki.ru|       463|
| 119.239.207.13|         China|         youtu.be|        51|
|  68.69.217.210|         China|        adobe.com|        10|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|
|   211.13.10.68|     Indonesia|          hud.gov|        29|
|   239.80.21.97|      Suriname|       smh.com.au|       218|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|
| 127.242.24.138|         China| surveymonkey.com|       123|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|
|   237.54.11.63|         China|       amazon.com|        83|
| 252.141.157.25|         Japan|      cornell.edu|       374|
|185.220.128.248|       Belgium|       weebly.com|       389|
|   151.77.19.45|   Afghanistan|independent.co.uk|       282|
|  9.161

In [106]:
# Add a column to say yes or no to whether the country is Mexico.
from pyspark.sql.functions import *
df = df.withColumn('is_Mexico', when(df.Country == 'Mexico', 'Yes').otherwise('No'))
df.show()

+---------------+--------------+-----------------+----------+---------+
|     ip_address|       Country|      Domain Name|Bytes_used|is_Mexico|
+---------------+--------------+-----------------+----------+---------+
|  52.81.192.172|         China| odnoklassniki.ru|       463|       No|
| 119.239.207.13|         China|         youtu.be|        51|       No|
|  68.69.217.210|         China|        adobe.com|        10|       No|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|       No|
|   211.13.10.68|     Indonesia|          hud.gov|        29|       No|
|   239.80.21.97|      Suriname|       smh.com.au|       218|       No|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|       No|
| 127.242.24.138|         China| surveymonkey.com|       123|       No|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|       No|
|   237.54.11.63|         China|       amazon.com|        83|       No|
| 252.141.157.25|         Japan|      cornell.edu|       374|   

In [108]:
# Group by your new column and sum bytes_used.
df1 = df.groupBy('is_Mexico').agg(sqlfunc.sum('Bytes_used').alias('total_bytes_used'))
df1.show()

+---------+----------------+
|is_Mexico|total_bytes_used|
+---------+----------------+
|       No|          508076|
|      Yes|            6293|
+---------+----------------+



In [109]:
# Group by country and use the sqlfunc.countDistinct function to calculate
# the number of IP address seen in EACH country.
import pyspark.sql.functions as sqlfunc
df1 = df.groupBy('Country').agg(sqlfunc.countDistinct('ip_address').alias('total_ip_address'))
df1.show()

+-----------+----------------+
|    Country|total_ip_address|
+-----------+----------------+
|       Chad|               1|
|     Russia|              56|
|   Paraguay|               1|
|      Yemen|               1|
|     Sweden|              28|
|Philippines|              65|
|   Malaysia|               5|
|     Turkey|               1|
|     Malawi|               2|
|    Germany|               5|
|    Comoros|               1|
|Afghanistan|               5|
|     Rwanda|               1|
|      Sudan|               1|
|     France|              21|
|     Greece|               8|
|  Sri Lanka|               3|
|   Dominica|               1|
|  Argentina|              14|
|    Belgium|               1|
+-----------+----------------+
only showing top 20 rows



In [110]:
df1 = df.groupBy('Country').agg(sqlfunc.countDistinct(df.ip_address).alias('total_ip_address'))
df1.sort(col('total_ip_address').desc()).show()

+--------------+----------------+
|       Country|total_ip_address|
+--------------+----------------+
|         China|             172|
|     Indonesia|             114|
|   Philippines|              65|
|        Russia|              56|
|        Brazil|              35|
|        Poland|              31|
|        Sweden|              28|
|         Japan|              25|
|Czech Republic|              23|
|      Portugal|              23|
|        France|              21|
|          Peru|              19|
|      Colombia|              17|
| United States|              15|
|       Ukraine|              14|
|     Argentina|              14|
|        Mexico|              13|
|      Thailand|              12|
|       Nigeria|              11|
|        Canada|              11|
+--------------+----------------+
only showing top 20 rows

